In [1]:
import json
import lawquery
import pandas as pd
import os
from tqdm import tqdm

# Load data

In [2]:
from underthesea import text_normalize,word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import string

def format_text(text,word_segmentation=False):
    text = re.sub(r'\s+', ' ', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    text = text_normalize(text)
    if word_segmentation:
        text = word_tokenize(text, format="text")
    return text

In [3]:
engines = {}
law_df = pd.read_csv('./documents/data.csv')
for i in range(len(law_df)):
    path = law_df['path'][i]
    so_hieu_van_ban = law_df['so_hieu_van_ban'][i]
    if so_hieu_van_ban not in engines:
        engines[so_hieu_van_ban] = lawquery.Engine(
            os.path.join('documents',path, 'tree.json.gz'),
        )
print("Number of engines:",len(engines))

documents = []
metadatas = []
ids = []

# take all
for engine in engines:
    results = engines[engine].query(node_type='điều')
    for result in results:
        documents.append(result.name+'\n'+result.content)
        metadatas.append({'law_id': engine, 'node_type': result.node_type, 'node_id': result.node_id})
        ids.append(result.id)

qa_df = pd.read_json('./answers_filtered.jsonl', lines=True, orient='records')
print("Number of documents:",len(documents))

Number of engines: 15
Number of documents: 740


# Create training triplet using TF-IDF


In [ ]:
tdidf_docs= [format_text(doc,True) for doc in documents]
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(tdidf_docs)

In [ ]:
instructor_for_q = 'Represent the legal question for retrieving evidence documents:'
instructor_for_r = 'Represent the legal document for retrieval:'

In [ ]:
datasets = []
to_run = len(qa_df)
for i in tqdm(range(to_run)):
    row = qa_df.iloc[i]
    data = {}
    query = row['cauhoi']
    pos = []
    for item in row['new_answers']:
        idx = metadatas.index(item)
        pos.append(documents[idx])
    query = format_text(query,True) 
    query_vector = tfidf.transform([query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    related_docs_indices = cosine_similarities.argsort()[:-100-1:-1]
    results = []
    for j in related_docs_indices:
        results.append(documents[j])
    neg = []
    for j in range(len(results)):
        if results[j] not in pos:
            neg.append(results[j])
        if len(neg) == len(pos):
            break

    for j in range(len(pos)):
        data = {}
        data['pos'] = [instructor_for_r,pos[j]]
        data['neg'] = [instructor_for_r,neg[j]]
        data['query'] = [instructor_for_q,query]
        data['task_name'] = 'tracuuluat'
        datasets.append(data)

100%|██████████| 4205/4205 [00:23<00:00, 176.80it/s]


In [ ]:
len(datasets)

6446

In [ ]:
import gzip

with gzip.open('../models/cachedir/medi-data.json.gz', 'wt', encoding='utf-8') as fout:
    fout.write(json.dumps(datasets, ensure_ascii=False, indent=4))

# Create training triplet using SBERT

In [7]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from InstructorEmbedding import INSTRUCTOR

In [16]:
chroma_client = chromadb.Client(Settings(chroma_api_impl="rest",
                                    chroma_server_host="localhost",
                                    chroma_server_http_port="8000",
                                    chroma_server_ssl_enabled=False
                                    ))
collection = chroma_client.get_or_create_collection(name="law_documents")

In [17]:
model = INSTRUCTOR('C:/Users/ngoph/Desktop/luanvan/model')
instructor_for_q = 'Represent the legal question for retrieving evidence documents:'
instructor_for_r = 'Represent the legal document for retrieval:'

load INSTRUCTOR_Transformer
max_seq_length  512


In [35]:
engines['2089/VBHN-BHXH'].query(node_type='điều',node_id='27')[0].content

'1. Cấp lại sổ BHXH do mất, hỏng, gộp sổ BHXH\n1.1. Thành phần hồ sơ:\na) Cấp lại sổ BHXH do mất, hỏng: Tờ khai tham gia, điều chỉnh thông tin BHXH, BHYT (Mẫu TK1-TS). b) Gộp sổ BHXH:\n- Tờ khai tham gia, điều chỉnh thông tin BHXH, BHYT (Mẫu TK1-TS).\n- Các sổ BHXH đề nghị gộp (nếu có);\n1.2. Số lượng hồ sơ: 01 bộ.\n2. Cấp lại sổ BHXH do thay đổi họ, tên, chữ đệm; ngày, tháng, năm sinh; giới tính, dân tộc; quốc tịch; điều chỉnh nội dung trên sổ BHXH\n2.1. Thành phần hồ sơ\na) Người tham gia\n- Tờ khai tham gia, điều chỉnh thông tin BHXH, BHYT (Mẫu TK1-TS).\n- Hồ sơ kèm theo (Mục 3,4 Phụ lục 01).\nb) Đơn vị: Bảng kê thông tin (Mẫu D01-TS).\n2.2. Số lượng hồ sơ: 01 bộ.\n3. Ghi xác nhận thời gian đóng BHXH cho người tham gia được cộng nối thời gian nhưng không phải đóng BHXH và điều chỉnh làm nghề hoặc công việc nặng nhọc, độc hại, nguy hiểm hoặc đặc biệt nặng nhọc, độc hại, nguy hiểm trước năm 1995\n3.1. Thành phần hồ sơ\na) Tờ khai tham gia, điều chỉnh thông tin BHXH, BHYT (Mẫu TK1-TS).

In [39]:
datasets = []
to_run = len(qa_df)
for i in tqdm(range(to_run)):
    row = qa_df.iloc[i]
    data = {}
    query = row['cauhoi']
    pos = []
    for item in row['new_answers']:
        idx = metadatas.index(item)
        pos.append(documents[idx])
    
    query_embeddings = model.encode([[instructor_for_q,query]],show_progress_bar=False).tolist()
    results = collection.query(query_embeddings=query_embeddings, n_results=100,include=["metadatas"])
    results = results['metadatas'][0]
    neg = []
    for result in results:
        doc = engines[result['law_id']].query(node_type=result['node_type'],node_id=result['node_id'])[0]
        doc = doc.name+'\n'+doc.content
        if doc not in pos:
            neg.append(doc)
        if len(neg) == len(pos):
            break

    for j in range(len(pos)):
        data = {}
        data['pos'] = [instructor_for_r,pos[j]]
        data['neg'] = [instructor_for_r,neg[j]]
        data['query'] = [instructor_for_q,query]
        data['task_name'] = 'tracuuluat'
        datasets.append(data)

100%|██████████| 4205/4205 [02:39<00:00, 26.34it/s]


In [40]:
len(datasets)

6446

In [43]:
with gzip.open('finetune-data-s2.json.gz', 'wt', encoding='utf-8') as fout:
    fout.write(json.dumps(datasets, ensure_ascii=False, indent=4))